In [1]:
import site, os
try:
    print(site.getsitepackages())
except:
    print(os.path.dirname(site.__file__) + '/site-packages')

/Users/csaintje/Documents/Recherche/Python/Envs/scientific_3_5/bin/../lib/python3.5/site-packages


In [2]:
import numpy as np
import scipy.integrate as sci
import scipy.optimize as sco
import theano.tensor as T
import theano
import itertools as it
import matplotlib
%matplotlib qt4
#print (matplotlib.rcsetup.interactive_bk)
#print (matplotlib.rcsetup.non_interactive_bk)
#print (matplotlib.rcsetup.all_backends)
print (matplotlib.get_backend())
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

Qt4Agg


Details for gaussian distribution as an exponential family

In [3]:
def s_1(x):
    return x

def s_2(x):
    return -x*x

def F_1D(theta1,theta2):
    return 0.25*theta1*theta1/theta2 + 0.5*np.log(np.pi) - 0.5*np.log(theta2) 

def gradF_1_1D(theta_1,theta_2):
    return 0.5*theta_1/theta_2

def gradF_2_1D(theta_1, theta_2):
    temp_1 = 0.5 / theta_2
    temp_2 = temp_1 * theta_1
    return -1. * (temp_2 * temp_2 + temp_1)

def gradF_1_nD(theta_1,theta_2):
    return 0.5*np.dot(np.inv(theta_2), theta_1)

def gradF_2_nD(theta_1, theta_2):
    temp_1 = 0.5*np.inv(theta_2)
    temp_2 = np.dot(temp1,theta_1)
    return - np.outer(temp_2,temp_2) - temp_1

def gradG_1_1D(eta_1,eta_2):
    return eta1 / (-eta_1 * eta1 - eta2)

def gradG_2_1D(eta_1,eta_2):
    return 0.5  / (-eta_1 * eta1 - eta2)

def gradG_1_nD(eta_1,eta_2):
    return np.dot(np.inv(-np.outer(eta_1, eta1) - eta2), eta1)

def gradG_2_nD(eta_1,eta_2):
    return 0.5*np.inv(-np.outer(eta_1, eta1) - eta2)


Cas 1D - Data

In [4]:
seed = 13
np.random.seed(seed)
N, batch_size = 2000, 100
mu_true, sigma_true = 1, 2
sigma2_true = sigma_true*sigma_true
theta1_true = mu_true / sigma2_true
theta2_true = 0.5 / sigma2_true
eta1_true = mu_true
eta2_true = - (mu_true*mu_true + sigma2_true)

X = np.random.normal(mu_true,np.sqrt(sigma2_true), N)

def batches(Y, bs):
    nb = np.int(np.ceil(len(Y) / bs))
    Yb = [Y[i * bs : (i+1) * bs] for i in range(nb)]
    def __temp(i):
        return Yb[i]
    return nb, Yb, __temp

Xt = X[: N//4]
Nt = len(Xt)
NtB, XtB, XtB_f = batches(Xt, batch_size)
Xv = X[N//4 : N//2]
Nv = len(Xv)
NvB, XvB, XvB_f = batches(Xv, batch_size)
Xtest = X[N//2:]
# print (Xt, XtB, NtB, XtB_f(1))

In [5]:
def ll(x, mu, sigma2):
    return -(x - mu)**2 /(2 * sigma2) - np.log(np.sqrt(2 * sigma2 * np.pi)) 

def ll_theta(x, theta1, theta2):
    return s_1(x) * theta1 + s_2(x) * theta2 - F_1D(theta1, theta2)

def ll_eta(x, eta1, eta2):
    theta1 = - eta1 / (eta1*eta1 + eta2)
    theta2 = - 0.5  / (eta1*eta1 + eta2)
    return ll_theta(x, theta1, theta2)
    

def ave_ll(mu, sigma2, chi):
    N = len(chi)
    return (1. / N) * sum(ll(x, mu, sigma2) for x in chi)
    
def C_N(mu, sigma2, chi):
    return -ave_ll(mu, sigma2, chi)

np.testing.assert_allclose(ll(4, mu_true, sigma2_true),
                          ll_theta(4, theta1_true, theta2_true))
np.testing.assert_allclose(ll(4, mu_true, sigma2_true),
                          ll_eta(4, eta1_true, eta2_true))

In [6]:
class ListTable(list):
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            for col in row:
                html.append("<td>{0}</td>".format(col))
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)
    
def disp_results(results):
    n = len(results)
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 5))
    colors = plt.cm.jet(np.linspace(0, 1, n))
    table = ListTable()
    table.append(['', 'Train', 'Test', 'mu', 'sigma^2'])
    table.append(['True', ave_ll(mu_true, sigma2_true, Xt), 
                  ave_ll(mu_true, sigma2_true, Xtest),
                  str(mu_true), str(sigma_true**2)])
    for c, result in enumerate(results):
        train, test, mu, sig, method = result
        axes[0,0].plot(train, color=colors[c], linewidth=1, linestyle='-', label=method)
        axes[0,0].set_title('Average likelihood on train set')
        axes[0,0].legend(loc=4)
        axes[0,1].plot(test, color=colors[c], linewidth=1, linestyle='-', label=method)
        axes[0,1].set_title('Average likelihood on test set')
        axes[0,1].legend(loc=4)
        axes[1,0].plot(mu, color=colors[c], linewidth=1, linestyle='-', label=method)
        axes[1,0].set_title('Estimates of $\mu$')
        axes[1,0].legend(loc=4)
        axes[1,1].plot(sig, color=colors[c], linewidth=1, linestyle='-', label=method)
        axes[1,1].set_title('Estimates of $\sigma^2$')
        axes[1,1].legend(loc=4)
        table.append([method, train[-1], test[-1],mu[-1], sig[-1]])
    display(HTML(table._repr_html_()))

# Points stationnaires de $C(\theta) = \mathbb{E}_{\pi} [C(\theta,x)] = \mathbb{E}_{\pi} [-\log p(x;\theta)]$ par Robbins-Monro


### Robbins-Monro

On dispose d'une fonction inconnue (supposée monotone) $M(\theta)$ telle que 
$$M(\theta) = \mathbb{E}_{\pi(\beta|\theta)} [\beta]$$ 
avec $\beta$ une v.a désignant des observations bruitées de $M(\theta)$.

On cherche la valeur $\theta^*$ telle que $M(\theta^*) = \alpha$.

Suite convergente de Robbins-Monro : $$\theta^{(t+1)} - \theta^{(t)} = a^{(t)} (\alpha - \beta^{(t)})$$

### Robbins-Monro  et gradient stochastique

$M(\theta) := \nabla_{\theta} C(\theta)$ est le gradient d'une fonction inconnue $C$. 

On cherche la valeur $\theta^*$ telle que $M(\theta^*) = \nabla C(\theta^*) = 0$.

Suite convergente de Robbins-Monro : $$\theta^{(t+1)} = \theta^{(t)} - a^{(t)} \beta^{(t)}$$
où $\beta^{(t)}$ est une observation bruitée de $\nabla_{\theta} C(\theta^{(t)})$.

### Pour notre cas
La fonction à minimiser est:
$$C(\theta) = \mathbb{E}_{\pi} [- \log p(x;\theta)]$$
où $\pi$ est la distribution inconnue dont on cherche une approximation $p$ paramétrée par $\theta$ (identique à la minimisation sur $\theta$ de $KL(\pi || p(.;\theta))$)

Son équivalent en discret:
$$C_N(\theta) = - N^{-1} \sum_i \log p(x_i;\theta)$$
avec $\lim_{N \rightarrow +\infty} C_N(\theta) = C(\theta)$ 

Sous conditions de regularité et dans la famille exponentielle, 
$$\nabla_{\theta} C(\theta) = \mathbb{E}_{\pi} [- \nabla_{\theta}\log p(x;\theta)]  = \mathbb{E}_{\pi} [- s(x) + \nabla_{\theta} F (\theta)]$$

Pour le relier à Robbin-Monro, on a une observation bruitée
$$\beta^{(t)} = - s(x^{(t)}) + \nabla_{\theta} F (\theta^{(t)})$$
et donc la suite convergente $$\theta^{(t+1)} = \theta^{(t)} + a^{(t)} (s(x^{(t)}) - \nabla_{\theta} F (\theta^{(t)}))$$

Est ce que la formulation suivante est équivalente ? sans doute non ...
$$\eta^{(t+1)} = \eta^{(t)} + a^{(t)} (s(x^{(t)}) - \eta^{(t)}))$$

Dans l'espace $H$ (paramètre d'espérance), la même optimisation:
$$C(\eta) = \mathbb{E}_{\pi} [- \log p(x;\eta)] = \mathbb{E}_{\pi} [B_{F^*}(s(x) : \eta) - F^*(s(x)) - k(x)]$$
$$\nabla_{\eta} C(\eta) = \mathbb{E}_{\pi} [\nabla_{\eta} B_{F^*}(s(x) : \eta)] = \mathbb{E}_{\pi} [\nabla_{\eta} (F^*(s(x)) - F^*(\eta) - <s(x) - \eta, \nabla_\eta F^*(\eta)>)]$$
$$ = \mathbb{E}_{\pi} [ - \nabla_{\eta} F^*(\eta) - [-1,0;0,-1]*\nabla_\eta F^*(\eta) - Hess F^*(\eta) (s(x) - \eta)]$$
$$ = \mathbb{E}_{\pi} [ - \nabla_{\eta} F^*(\eta) + \nabla_{\eta} F^*(\eta) - Hess F^*(\eta) (s(x) - \eta)] = \mathbb{E}_{\pi} [- Hess F^*(\eta) (s(x) - \eta)]$$
et donc la suite convergente 
$$\eta^{(t+1)} = \eta^{(t)} + a^{(t)} Hess F^*(\eta^{(t)}) (s(x^{(t)}) - \eta^{(t)})$$

In [7]:
from sympy import Function, Derivative, var, simplify
from sympy.abc import x, y, z, t
from sympy import diff, log, pi
s1 = Function("s1")(x,y)
s2 = Function("s2")(x,y)
F = Function("F")(z,t)
F1 = Derivative(F, z)
F2 = Derivative(F, t)
expr = -((s1 - z) * F1 + (s2 - t)*F2)
print (diff(expr, z))
print (diff(expr, t))
x, eta1, eta2 = var('x eta1 eta2')
s1 = x
s2 = -x*x
ld = log(2*(- eta1*eta1 - eta2))
F =  - 0.5 * (1 + log(pi) + ld)
dF1 = diff(F, eta1)
dF2 = diff(F, eta2)
dF11 = diff(dF1, eta1)
dF12 = diff(dF1, eta2)
dF21 = diff(dF2, eta1)
dF22 = diff(dF2, eta2)
print (dF11)
print (dF12)
print (dF21)
print (dF22)
print (-simplify(dF11*(s1-eta1)+dF12*(s2-eta2)))
print (-simplify(dF21*(s1-eta1)+dF22*(s2-eta2)))

-(-t + s2(x, y))*Derivative(F(z, t), t, z) - (-z + s1(x, y))*Derivative(F(z, t), z, z) + Derivative(F(z, t), z)
-(-t + s2(x, y))*Derivative(F(z, t), t, t) - (-z + s1(x, y))*Derivative(F(z, t), t, z) + Derivative(F(z, t), t)
8.0*eta1**2/(-2*eta1**2 - 2*eta2)**2 + 2.0/(-2*eta1**2 - 2*eta2)
4.0*eta1/(-2*eta1**2 - 2*eta2)**2
4.0*eta1/(-2*eta1**2 - 2*eta2)**2
2.0/(-2*eta1**2 - 2*eta2)**2
(1.0*eta1*(eta2 + x**2) + (eta1 - x)*(8.0*eta1**2 - 8.0*eta2)/8)/(eta1**2 + eta2)**2
(1.0*eta1*(eta1 - x) + 0.5*eta2 + 0.5*x**2)/(eta1**2 + eta2)**2


# Debut des manips

In [8]:
results = []

## Points stationnaires de $C_N(\theta)$ par dérivation exacte

   $$\nabla C_N(\theta) = 0 \equiv -N^{-1} \sum_i (s(x_i) - \nabla F(\theta)) = 0 \equiv  \nabla F(\theta) = N^{-1} \sum_i s(x_i)$$

In [9]:
gradF_pt_stat_1 = np.sum(s_1(x) for x in Xt) / Nt
gradF_pt_stat_2 = np.sum(s_2(x) for x in Xt) / Nt
pt_stat = gradF_pt_stat_1, - gradF_pt_stat_1**2 - gradF_pt_stat_2
print ((mu_true, sigma2_true), ' vs ', pt_stat)
print('average ll on training set : ', 
      ave_ll(mu_true, sigma2_true, Xt), ' vs ',
      ave_ll(pt_stat[0], pt_stat[1], Xt))
print('average ll on test set : ', 
      ave_ll(mu_true, sigma2_true, Xtest), ' vs ',
      ave_ll(pt_stat[0], pt_stat[1], Xtest))
gradF_pt_stat_1 = np.cumsum([s_1(x) for x in Xt]) / np.arange(1,Nt+1)
gradF_pt_stat_2 = np.cumsum([s_2(x) for x in Xt]) / np.arange(1,Nt+1)
mu_list = gradF_pt_stat_1
sig_list = -gradF_pt_stat_1**2 -gradF_pt_stat_2
ave_ll_train = [ave_ll(mu, sig, Xt) for mu,sig in zip(mu_list, sig_list)]
ave_ll_test = [ave_ll(mu, sig, Xtest) for mu,sig in zip(mu_list, sig_list)]
this_result = [ave_ll_train, ave_ll_test, mu_list, sig_list, 'Exact']
results.append(this_result)

(1, 4)  vs  (0.93141726306687489, 3.5299590612896603)
average ll on training set :  -2.0539185454  vs  -2.04958166996
average ll on test set :  -2.10152922778  vs  -2.10479830458


In [10]:
disp_results([this_result])

,Train,Test,mu,sigma^2
True,-2.0539185454014794,-2.101529227781853,1,4
Exact,-2.0495816699594678,-2.104798304581388,0.9314172630668749,3.5299590612896603


# Optimization via scipy

In [11]:
def fun_C_N(mu_sigma2):
    return C_N(mu_sigma2[0], mu_sigma2[1], Xt)
np.random.rand(seed)
x0 = (np.random.randn(), np.random.random())
bnds = ((-np.inf, np.inf), (1e-6, np.inf))   # variance is positive
res_C_N = sco.minimize(fun_C_N, x0, bounds=bnds) #, options={'gtol': 1e-6, 'disp': True})
print(res_C_N)
print('average ll on training set : ', 
      ave_ll(mu_true, sigma2_true, Xt), ' vs ',
      ave_ll(res_C_N.x[0], res_C_N.x[1], Xt))
print('average ll on test set : ', 
      ave_ll(mu_true, sigma2_true, Xtest), ' vs ',
      ave_ll(res_C_N.x[0], res_C_N.x[1], Xtest))

      fun: 2.049581669969299
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([  1.06581410e-06,  -7.10542736e-07])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 45
      nit: 13
   status: 0
  success: True
        x: array([ 0.93142087,  3.52993911])
average ll on training set :  -2.0539185454  vs  -2.04958166997
average ll on test set :  -2.10152922778  vs  -2.10479855013


## Optimization via Stochastic Gradient Descent  (natural space)
mostly fail:
- $\alpha = 0.01$

success:
- $\alpha = 0.001$

In [12]:
np.random.seed(seed)

alpha = 0.001
epochs = 10

mu_0, sigma2_0 = np.random.randn(), np.random.random()
theta1, theta2 = mu_0 / sigma2_0, 0.5 / sigma2_0

def grad_nll_1(x, theta1, theta2):
    return - (s_1(x) - gradF_1_1D(theta1, theta2))

def grad_nll_2(x, theta1, theta2):
    return - (s_2(x) - gradF_2_1D(theta1, theta2))
    
mu_list, sig_list, ave_ll_list_train, ave_ll_list_test = [], [], [], []
for __ in range(epochs):
    for xt in Xt:
        #print("avant theta",theta1, theta2, " x ", xt)
        #print ("grad 1 ", gradF_1_1D(theta1, theta2), grad_nll_1(xt, theta1, theta2))
        #print ("grad 2 ", gradF_2_1D(theta1, theta2), grad_nll_2(xt, theta1, theta2))
        theta1 -= alpha*grad_nll_1(xt, theta1, theta2)
        theta2 -= alpha*grad_nll_2(xt, theta1, theta2)
        #print("apres theta", theta1, theta2)
        mu_est, sigma2_est = 0.5*theta1/theta2, 0.5/theta2
        #print("apres mu_sd2",   mu_est, sigma2_est)
        mu_list.append(mu_est)
        sig_list.append(sigma2_est)
        ave_ll_list_train.append(ave_ll(mu_est, sigma2_est, Xt)) 
        ave_ll_list_test.append(ave_ll(mu_est, sigma2_est, Xtest)) 
this_result = [ave_ll_list_train, ave_ll_list_test, mu_list, sig_list, 'SGD(T)-'+str(alpha)]
results.append(this_result)

In [13]:
#disp_results([this_result])
disp_results(results)

,Train,Test,mu,sigma^2
True,-2.0539185454014794,-2.101529227781853,1,4
Exact,-2.0495816699594678,-2.104798304581388,0.9314172630668749,3.5299590612896603
SGD(T)-0.001,-2.07042166281291,-2.1441114500583507,0.7238246892043892,2.828639631552067


## Optimization via Stochastic Gradient Descent  

Il s'agit d'un réécriture simple en remplacant $\nabla F$ par $\eta$. 

Cela ressemble dans la forme à l'approximation stochastique du Online EM.

L'algorihme converge car les deux optimisations (avec celles du dessus) sont liées. 

(J'ai fait le calcul $\eta^{(n+1)} - \eta^{(n)}$ vers $\theta^{(n+1)} - \theta^{(n)}$)

Il ne correspond pas à un SGD dand $H$.

In [14]:
np.random.seed(seed)

alpha = 0.001
epochs = 10
mu_0, sigma2_0 = np.random.randn(), np.random.random()
eta1, eta2 = mu_0 , -(mu_0*mu_0 + sigma2_0)
def grad_nll_1(x, eta1, eta2):
    return -(s_1(x) - eta1)

def grad_nll_2(x, eta1, eta2):
    return -(s_2(x) - eta2)
    
mu_list, sig_list, ave_ll_list_train, ave_ll_list_test = [], [], [], []
for __ in range(epochs):
    for x in Xt:
        #print("avant eta",eta1, eta2, " x ", x)
        #print ("grad 1 ", grad_nll_1(x, eta1, eta2))
        #print ("grad 2 ", grad_nll_2(x, eta1, eta2))
        eta1 -= alpha*grad_nll_1(x, eta1, eta2)
        eta2 -= alpha*grad_nll_2(x, eta1, eta2)
        #print("apres eta", eta1, eta2)
        mu_est, sigma2_est = eta1, -(eta1*eta1+eta2)
        #print("apres mu_sd2", mu_est, sd_est)
        mu_list.append(mu_est)
        sig_list.append(sigma2_est)
        ave_ll_list_train.append(ave_ll(mu_est, sigma2_est, Xt)) 
        ave_ll_list_test.append(ave_ll(mu_est, sigma2_est, Xtest))
this_result = [ave_ll_list_train, ave_ll_list_test, mu_list, sig_list, 'SGD(M)-'+str(alpha)]
results.append(this_result)

In [15]:
#disp_results([this_result])
disp_results(results)

,Train,Test,mu,sigma^2
True,-2.0539185454014794,-2.101529227781853,1,4
Exact,-2.0495816699594678,-2.104798304581388,0.9314172630668749,3.5299590612896603
SGD(T)-0.001,-2.07042166281291,-2.1441114500583507,0.7238246892043892,2.828639631552067
SGD(M)-0.001,-2.049614040072827,-2.1049867419813966,0.9165525945786573,3.5375054005352187


## Optimization via Stochastic Gradient Descent  (Expectation space)

In [22]:
np.random.seed(seed)

alpha = 0.1
epochs = 10
mu_0, sigma2_0 = np.random.randn(), np.random.random()
eta1, eta2 = mu_0 , -(mu_0*mu_0 + sigma2_0)
def grad_1(x, eta1, eta2):
    return (1.0*eta1*(eta2 + x**2) + (eta1 - x)*(8.0*eta1**2 - 8.0*eta2)/8.)/(eta1**2 + eta2)**2

def grad_2(x, eta1, eta2):
    return (1.0*eta1*(eta1 - x) + 0.5*eta2 + 0.5*x**2)/(eta1**2 + eta2)**2
    
mu_list, sig_list, ave_ll_list_train, ave_ll_list_test = [], [], [], []
for __ in range(epochs):
    for x in Xt:
        #print("avant eta",eta1, eta2, " x ", x)
        #print ("grad 1 ", grad_nll_1(x, eta1, eta2))
        #print ("grad 2 ", grad_nll_2(x, eta1, eta2))
        eta1 -= alpha*grad_1(x, eta1, eta2)
        eta2 -= alpha*grad_2(x, eta1, eta2)
        #print("apres eta", eta1, eta2)
        mu_est, sigma2_est = eta1, -(eta1*eta1+eta2)
        #print("apres mu_sd2", mu_est, sd_est)
        mu_list.append(mu_est)
        sig_list.append(sigma2_est)
        ave_ll_list_train.append(ave_ll(mu_est, sigma2_est, Xt)) 
        ave_ll_list_test.append(ave_ll(mu_est, sigma2_est, Xtest))
this_result = [ave_ll_list_train, ave_ll_list_test, mu_list, sig_list, 'SGD(H)-'+str(alpha)]
results.append(this_result)

In [23]:
#disp_results([this_result])
disp_results(results)

,Train,Test,mu,sigma^2
True,-2.0539185454014794,-2.101529227781853,1,4
Exact,-2.0495816699594678,-2.104798304581388,0.9314172630668749,3.5299590612896603
SGD(T)-0.001,-2.07042166281291,-2.1441114500583507,0.7238246892043892,2.828639631552067
SGD(M)-0.001,-2.049614040072827,-2.1049867419813966,0.9165525945786573,3.5375054005352187
SGD(H)-0.001,-2.1278333040960002,-2.221087705468255,0.5440596567516236,2.3608284762100604
SGD(H)-0.01,-2.051467215068424,-2.111436864066219,0.843550539862887,3.3456754372275137
SGD(H)-0.1,-2.059234212658935,-2.119884544331419,0.6721535444025877,3.492304178578393


# Theano

In [18]:
def gradient_updates(cost, params, alpha):
    updates = [(param, param - alpha*T.grad(cost, param)) for param in params]
    return updates

def gradient_updates_momentum(cost, params, alpha, momentum):
    """
    http://caffe.berkeleyvision.org/tutorial/solver.html
    """
    assert momentum < 1 and momentum >= 0
    updates = []
    for param in params:
        V = theano.shared(param.get_value()*0.)
        updates.append((param, param + V))
        updates.append((V, momentum*V - alpha * T.grad(cost, param)))
    return updates

# on peut rajouter d'autres méthodes ici

def gradient_updates_adam(cost, params, alpha, beta1, beta2):
    assert beta1 < 1 and beta1 >= 0
    assert beta2 < 1 and beta2 >= 0
    updates = []
    t = theano.shared(1.)
    updates.append((t, t+1))
    for param in params:
        gt = T.grad(cost, param) 
        mt = theano.shared(0.)
        updates.append((mt, beta1 * mt + (1-beta1) * gt))
        vt = theano.shared(0.)
        updates.append((vt, beta2 * vt + (1-beta2) * gt * gt))
        alpha_t = theano.shared(0.)
        updates.append((alpha_t, 
                        alpha*T.sqrt(1 - beta2**t)/(1 - beta1**t)))
        updates.append((param, param - alpha_t*mt/(T.sqrt(vt + 1e-8))))
    return updates   

## Optimization SGD via Theano (source space)

In [19]:
np.random.seed(seed)

alpha = 0.001
epochs = 10
# Declare Theano symbolic variables
x = T.scalar()
mu_0, sigma2_0 = np.random.randn(), np.random.random()
mu = theano.shared(mu_0, name="mu")
sigma2 = theano.shared(sigma2_0, name="sigma2") # ensure initial value is positive

# Construct Theano expression graph to minimize (- log-likehood for univariate gaussian)
nll =  (x - mu)**2 /(2 * sigma2) + T.log(T.sqrt(2 * sigma2 *np.pi))

# Compile
train = theano.function(
          inputs=[x],
          updates=gradient_updates(nll, [mu, sigma2], alpha))

# Train
mu_list, sig_list, ave_ll_list_train, ave_ll_list_test = [], [], [], []
for i in range(epochs):
    for xt in Xt:
        __ = train(xt)
        mu_est, sigma2_est = mu.get_value(), sigma2.get_value()
        mu_list.append(mu_est)
        sig_list.append(sigma2_est)
        ave_ll_list_train.append(ave_ll(mu_est, sigma2_est, Xt)) 
        ave_ll_list_test.append(ave_ll(mu_est, sigma2_est, Xtest)) 
this_result = [ave_ll_list_train, ave_ll_list_test, mu_list, sig_list, 'SGD(S)-'+str(alpha)]
results.append(this_result)

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '20959' (I am process '21026')
00001	#include <Python.h>
00002	#include <iostream>
00003	#include "theano_mod_helper.h"
00004	#include <math.h>
00005	#include <numpy/arrayobject.h>
00006	#include <numpy/arrayscalars.h>
00007	//////////////////////
00008	////  Support Code
00009	//////////////////////
00010	
00011	
00012	    namespace {
00013	    struct __struct_compiled_op_m3d3273bf5f2a3791c5d3a5b9dcafd92d {
00014	        PyObject* __ERROR;
00015	
00016	        PyObject* storage_V3;
00017	PyObject* storage_V1;
00018	        
00019	
00020	        __struct_compiled_op_m3d3273bf5f2a3791c5d3a5b9dcafd92d() {
00021	            // This is only somewhat safe because we:
00022	            //  1) Are not a virtual class
00023	            //  2) Do not use any virtual classes in the members
00024	            //  3) Deal with mostly POD and pointers
00025	
00026	            // If this changes, we would have to revise this, 

clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-tbm'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-fma4'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-prfchw'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-pcommit'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-cl

00001	#include <Python.h>
00002	#include <iostream>
00003	#include "theano_mod_helper.h"
00004	#include <math.h>
00005	#include <numpy/arrayobject.h>
00006	#include <numpy/arrayscalars.h>
00007	//////////////////////
00008	////  Support Code
00009	//////////////////////
00010	
00011	
00012	    namespace {
00013	    struct __struct_compiled_op_m3d3273bf5f2a3791c5d3a5b9dcafd92d {
00014	        PyObject* __ERROR;
00015	
00016	        PyObject* storage_V3;
00017	PyObject* storage_V1;
00018	        
00019	
00020	        __struct_compiled_op_m3d3273bf5f2a3791c5d3a5b9dcafd92d() {
00021	            // This is only somewhat safe because we:
00022	            //  1) Are not a virtual class
00023	            //  2) Do not use any virtual classes in the members
00024	            //  3) Deal with mostly POD and pointers
00025	
00026	            // If this changes, we would have to revise this, but for
00027	            // now I am tired of chasing segfaults because
00028	            // initializati

clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-tbm'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-fma4'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-prfchw'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-pcommit'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-cl

00001	#include <Python.h>
00002	#include <iostream>
00003	#include "theano_mod_helper.h"
00004	#include <math.h>
00005	#include <numpy/arrayobject.h>
00006	#include <numpy/arrayscalars.h>
00007	//////////////////////
00008	////  Support Code
00009	//////////////////////
00010	
00011	
00012	    namespace {
00013	    struct __struct_compiled_op_m3d3273bf5f2a3791c5d3a5b9dcafd92d {
00014	        PyObject* __ERROR;
00015	
00016	        PyObject* storage_V3;
00017	PyObject* storage_V1;
00018	        
00019	
00020	        __struct_compiled_op_m3d3273bf5f2a3791c5d3a5b9dcafd92d() {
00021	            // This is only somewhat safe because we:
00022	            //  1) Are not a virtual class
00023	            //  2) Do not use any virtual classes in the members
00024	            //  3) Deal with mostly POD and pointers
00025	
00026	            // If this changes, we would have to revise this, but for
00027	            // now I am tired of chasing segfaults because
00028	            // initializati

clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-tbm'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-fma4'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-prfchw'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-pcommit'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-target-feature'
clang: error: unknown argument: '-cl

00001	#include <Python.h>
00002	#include <iostream>
00003	#include "theano_mod_helper.h"
00004	#include <math.h>
00005	#include <numpy/arrayobject.h>
00006	#include <numpy/arrayscalars.h>
00007	#include <vector>
00008	#include <algorithm>
00009	//////////////////////
00010	////  Support Code
00011	//////////////////////
00012	
00013	
00014	    namespace {
00015	    struct __struct_compiled_op_m39a994c45ddc56b4e042d425f27b382f {
00016	        PyObject* __ERROR;
00017	
00018	        PyObject* storage_V3;
00019	PyObject* storage_V5;
00020	PyObject* storage_V1;
00021	        
00022	
00023	        __struct_compiled_op_m39a994c45ddc56b4e042d425f27b382f() {
00024	            // This is only somewhat safe because we:
00025	            //  1) Are not a virtual class
00026	            //  2) Do not use any virtual classes in the members
00027	            //  3) Deal with mostly POD and pointers
00028	
00029	            // If this changes, we would have to revise this, but for
00030	            /

Exception: ('The following error happened while compiling the node', Elemwise{sub,no_inplace}(<TensorType(float64, scalar)>, mu), '\n', "Compilation failed (return status=1): clang: error: unknown argument: '-target-feature'. clang: error: unknown argument: '-target-feature'. clang: error: unknown argument: '-target-feature'. clang: error: unknown argument: '-tbm'. clang: error: unknown argument: '-target-feature'. clang: error: unknown argument: '-target-feature'. clang: error: unknown argument: '-target-feature'. clang: error: unknown argument: '-fma4'. clang: error: unknown argument: '-target-feature'. clang: error: unknown argument: '-prfchw'. clang: error: unknown argument: '-target-feature'. clang: error: unknown argument: '-target-feature'. clang: error: unknown argument: '-target-feature'. clang: error: unknown argument: '-target-feature'. clang: error: unknown argument: '-target-feature'. clang: error: unknown argument: '-target-feature'. clang: error: unknown argument: '-pcom

# Optimization SGD via Theano (natural space)

Fails for $\alpha =0.005$

In [ ]:
np.random.seed(seed)

alpha = 0.001
epochs = 10

# Declare Theano symbolic variables
x = T.scalar()

mu_0, sigma2_0 = np.random.randn(), np.random.random() 
theta1 = theano.shared(mu_0 / sigma2_0, name="theta1") # ensure initial value is positive
theta2 = theano.shared(0.5 / sigma2_0, name="theta2") # ensure initial value is positive

# Construct Theano expression graph to minimize (- log-likehood for univariate gaussian)
nll_theta =  -(x*theta1 - theta2*x*x - 0.25*theta1*theta1/theta2 - 0.5*np.log(np.pi) +0.5*T.log(theta2))

# Compile
train = theano.function(
          inputs=[x],
          updates=gradient_updates(nll_theta, [theta1, theta2], alpha))

# Train
mu_list, sig_list, ave_ll_list_train, ave_ll_list_test = [], [], [], []
for i in range(epochs):
    for xt in Xt:
        train(xt)
        mu_est = 0.5*theta1.get_value()/theta2.get_value()
        sigma2_est = 0.5/theta2.get_value()
        mu_list.append(mu_est)
        sig_list.append(sigma2_est)
        ave_ll_list_train.append(ave_ll(mu_est, sigma2_est, Xt)) 
        ave_ll_list_test.append(ave_ll(mu_est, sigma2_est, Xtest))
this_result = [ave_ll_list_train, ave_ll_list_test, mu_list, sig_list, 'SGD(T)-'+str(alpha)]
results.append(this_result)

# Optimization SGD via Theano (expectation space)

In [ ]:
np.random.seed(seed)

alpha = 0.001
epochs = 10

# Declare Theano symbolic variables
x = T.scalar()

mu_0, sigma2_0 = np.random.randn(), np.random.random()
eta1 = theano.shared(mu_0, name="eta1") # ensure initial value is positive
eta2 = theano.shared(-(mu_0*mu_0 + sigma2_0), name="eta2") # ensure initial value is positive
print(eta1.get_value(), eta2.get_value())
# Construct Theano expression graph to minimize (- log-likehood for univariate gaussian)
theta1 = - eta1 / (eta1*eta1 + eta2)
theta2 = - 0.5  / (eta1*eta1 + eta2)
nll_eta =  -(x*theta1 - theta2*x*x - 0.25*theta1*theta1/theta2 - 0.5*np.log(np.pi) +0.5*T.log(theta2))

# Compile
train = theano.function(
          inputs=[x],
          updates=gradient_updates(nll_eta, [eta1, eta2], alpha))

# Train
mu_list, sig_list, ave_ll_list_train, ave_ll_list_test = [], [], [], []
for i in range(epochs):
    for xt in Xt:
        train(xt)
        # print(g1, s_1(xt) - eta1.get_value())
        # print(g2, s_2(xt) - eta2.get_value())
        mu_est = eta1.get_value()
        sigma2_est = - (eta1.get_value()*eta1.get_value()+eta2.get_value())
        mu_list.append(mu_est)
        sig_list.append(sigma2_est)
        ave_ll_list_train.append(ave_ll(mu_est, sigma2_est, Xt)) 
        ave_ll_list_test.append(ave_ll(mu_est, sigma2_est, Xtest))
this_result = [ave_ll_list_train, ave_ll_list_test, mu_list, sig_list, 'SGD(H)-'+str(alpha)]
results.append(this_result)

## Optimization via Theano +  SGD Nesterov momemtum (Source space)

In [ ]:
np.random.seed(seed)

alpha = 0.01
momentum = 0.9
epochs = 10
# Declare Theano symbolic variables
x = T.scalar()
mu_0, sigma2_0 = np.random.randn(), np.random.random()
mu = theano.shared(mu_0, name="mu")
sigma2 = theano.shared(sigma2_0, name="sigma2") # ensure initial value is positive

# Construct Theano expression graph to minimize (- log-likehood for univariate gaussian)
nll =  (x - mu)**2 /(2 * sigma2) + T.log(T.sqrt(2 * sigma2 *np.pi))

# Compile
train = theano.function(
          inputs=[x],
          updates=gradient_updates_momentum(nll, [mu, sigma2], alpha, momentum))

# Train
mu_list, sig_list, ave_ll_list_train, ave_ll_list_test = [], [], [], []
for i in range(epochs):
    for xt in Xt:
        train(xt)
        mu_est, sigma2_est = mu.get_value(), sigma2.get_value()
        mu_list.append(mu_est)
        sig_list.append(sigma2_est)
        ave_ll_list_train.append(ave_ll(mu_est, sigma2_est, Xt)) 
        ave_ll_list_test.append(ave_ll(mu_est, sigma2_est, Xtest))
this_result = [ave_ll_list_train, ave_ll_list_test, mu_list, sig_list, 'Mom. SGD(S)-'+str(momentum)+','+str(alpha)]
results.append(this_result)

## Optimization via Theano +  ADAM (Source space)

In [ ]:
np.random.seed(seed)

alpha = 0.005
beta1 = 0.9
beta2 = 0.995
epochs = 10
# Declare Theano symbolic variables
x = T.scalar()
mu_0, sigma2_0 = np.random.randn(), np.random.random()
mu = theano.shared(mu_0, name="mu")
sigma2 = theano.shared(sigma2_0, name="sigma2") # ensure initial value is positive

# Construct Theano expression graph to minimize (- log-likehood for univariate gaussian)
nll =  (x - mu)**2 /(2 * sigma2) + T.log(T.sqrt(2 * sigma2 *np.pi))

# Compile
train = theano.function(
          inputs=[x],
          updates=gradient_updates_adam(nll, [mu, sigma2], 
                                        alpha, beta1, beta2))

# Train
mu_list, sig_list, ave_ll_list_train, ave_ll_list_test = [], [], [], []
for i in range(epochs):
    for xt in Xt:
        train(xt)
        mu_est, sigma2_est = mu.get_value(), sigma2.get_value()
        mu_list.append(mu_est)
        sig_list.append(sigma2_est)
        ave_ll_list_train.append(ave_ll(mu_est, sigma2_est, Xt)) 
        ave_ll_list_test.append(ave_ll(mu_est, sigma2_est, Xtest)) 
this_result = [ave_ll_list_train, ave_ll_list_test, mu_list, sig_list, 'Adam(S)-'+str(beta1)+','+str(beta2)+','+str(alpha)]
results.append(this_result)

In [ ]:
disp_results(results)

# Optimization via downhill (adam)

Pas encore terminé...

In [ ]:
climate.enable_default_logging()

np.random.seed(seed)

alpha = 0.01
momentum = 0.9
epochs = 10
# Declare Theano symbolic variables
x = T.vector()
mu_0, sigma2_0 = np.random.randn(), np.random.random()
mu = theano.shared(mu_0, name="mu")
sigma2 = theano.shared(sigma2_0, name="sigma2") # ensure initial value is positive

# Construct Theano expression graph to minimize (- log-likehood for univariate gaussian)
#nll =  (x - mu)**2 /(2 * sigma2) + T.log(T.sqrt(2 * sigma2 *np.pi))
navell =  T.sum(T.sqr(x - mu) /(2 * sigma2) + T.log(T.sqrt(2 * sigma2 *np.pi)))
train = downhill.Dataset(Xt)
#valid = downhill.Dataset(Xtest)

downhill.minimize(
    loss=navell,
    train=Xt,
    patience=0,
    max_gradient_norm=1,          # Prevent gradient explosion!
    learning_rate=alpha)


print(mu.get_value())
print(sigma2.get_value())

In [ ]:
help(downhill.minimize)

## L'exemple de la doc de downhill ne marche pas

In [ ]:
climate.enable_default_logging()


import theano.tensor as TT
def rand(a, b): return np.random.randn(a, b).astype('f')

A, B, K = 20, 5, 3

# Set up a matrix factorization problem to optimize.
u = theano.shared(rand(A, K), name='u')
v = theano.shared(rand(K, B), name='v')
e = TT.sqr(TT.matrix() - TT.dot(u, v))

# Minimize the regularized loss with respect to a data matrix.
y = np.dot(rand(A, K), rand(K, B)) + rand(A, B)

downhill.minimize(
    loss=e.mean() + abs(u).mean() + (v * v).mean(),
    train=[y],
    patience=0,
    batch_size=A,                 # Process y as a single batch.
    max_gradient_norm=1,          # Prevent gradient explosion!
    learning_rate=0.1,
    monitors=(('err', e.mean()),  # Monitor during optimization.
              ('|u|<0.1', (abs(u) < 0.1).mean()),
              ('|v|<0.1', (abs(v) < 0.1).mean())),
    monitor_gradients=True)

# Print out the optimized coefficients u and basis v.
print('u =', u.get_value())
print('v =', v.get_value())

In [ ]:
num2str